In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical

# Define the paths to your dataset directories
dataset_dir = 'dataset'  # Update this path to your dataset
real_dir = os.path.join(dataset_dir, 'real')
default_dir = os.path.join(dataset_dir, 'default')
fake_dir = os.path.join(dataset_dir, 'fake')

# Initialize lists to hold image data and labels
train_images = []
train_labels = []

# Define a function to load and preprocess the image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to 224x224
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add an extra dimension for batch size
    img_array = preprocess_input(img_array)  # Normalize the image
    return img_array

# Define a function to load images and assign labels
def load_images_from_directory(directory, label):
    for img_file in os.listdir(directory):
        img_path = os.path.join(directory, img_file)
        # Check if the path is a file and has an appropriate image extension
        if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            try:
                img = load_and_preprocess_image(img_path)  # Load and preprocess the image
                train_images.append(img)  # Append the preprocessed image to the list
                train_labels.append(label)  # Append the corresponding label (integer)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")  # Print the error message




In [3]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define batch size and directories
batch_size = 16
train_dir = 'dataset/train/'
validation_dir = 'dataset/validation/'
augmented_train_dir = 'dataset/augmented_train/'  # Directory to save augmented images

# Create the directory to save augmented images if it doesn't exist
if not os.path.exists(augmented_train_dir):
    os.makedirs(augmented_train_dir)

# Create an instance of ImageDataGenerator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data should only rescale
val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    save_to_dir=augmented_train_dir,  # Save augmented images here
    save_prefix='aug',  # Prefix for saved images
    save_format='jpeg'  # Format for saved images
)

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 126 images belonging to 3 classes.
Found 158 images belonging to 3 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import regularizers
from sklearn.utils import class_weight
import numpy as np

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being trained
base_model.trainable = False

# Create model with regularization and dropout
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  # Reduced dense layer size
    tf.keras.layers.Dropout(0.5),  # Adjusted dropout rate
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: Real, Default, Suspicious
])

# Set a lower learning rate
learning_rate = 0.0001  # Adjust as necessary
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_images_count = 126
validation_images_count = 158 
batch_size = 8

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Convert class weights to a dictionary
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Fit the model with class weights
model.fit(
    train_generator,
    steps_per_epoch=train_images_count // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_images_count // batch_size,
    epochs=15,
    class_weight=class_weight_dict  # Pass the class weights here
)



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 23s 2us/step
Epoch 1/15


c:\Users\shriv\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 8/15 ━━━━━━━━━━━━━━━━━━━━ 2s 298ms/step - accuracy: 0.2083 - loss: 6.0053

c:\Users\shriv\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 564ms/step - accuracy: 0.2592 - loss: 5.8714 - val_accuracy: 0.6835 - val_loss: 5.0932
Epoch 2/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 356ms/step - accuracy: 0.5880 - loss: 5.1292 - val_accuracy: 0.7278 - val_loss: 4.8607
Epoch 3/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 344ms/step - accuracy: 0.6518 - loss: 4.8944 - val_accuracy: 0.7278 - val_loss: 4.6994
Epoch 4/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 338ms/step - accuracy: 0.7611 - loss: 4.7040 - val_accuracy: 0.7405 - val_loss: 4.6202
Epoch 5/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 6s 322ms/step - accuracy: 0.7301 - loss: 4.6242 - val_accuracy: 0.7468 - val_loss: 4.5637
Epoch 6/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 332ms/step - accuracy: 0.7972 - loss: 4.5062 - val_accuracy: 0.7658 - val_loss: 4.5183
Epoch 7/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 325ms/step - accuracy: 0.8322 - loss: 4.4124 - val_accuracy: 0.7911 - val_loss: 4.4696
Epoch 8/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 7s 324ms/step - accuracy: 0.8276 - loss: 4.3527 - val_accuracy: 0.7722 - val

In [5]:

def predict_profile_picture(img_path):
    img = load_and_preprocess_image(img_path)
    predictions = model.predict(img)
    class_idx = np.argmax(predictions, axis=1)[0]
    
    if class_idx == 0:
        return 'default'
    elif class_idx == 1:
        return 'fake'
    else:
        return 'Real'

# Example usage:
profile_pic = '_.aishvarya_profile_pic.jpg.jpg'
label = predict_profile_picture(profile_pic)
print(f"The profile picture is classified as: {label}")

FileNotFoundError: [Errno 2] No such file or directory: '_.aishvarya_profile_pic.jpg.jpg'

In [6]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'default': 0, 'fake': 1, 'real': 2}
{'default': 0, 'fake': 1, 'real': 2}
